In [ ]:
%cd /home/stefano/dev/active/spreads-arb
%load_ext line_profiler

import itertools
import logging
import os
import pickle
from datetime import date, datetime, timedelta

import cryptomart as cm
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import requests
import vectorbt as vbt

import app

identifier = "spreads-arb-v2"

In [ ]:
ohlcvs = app.data_prep.all_ohlcv("2023-02-01", "2023-05-04", "interval_1h", refresh=False, identifiers=[identifier])

In [ ]:
ohlcvs = ohlcvs[ohlcvs.missing_rows <= 0]
ohlcvs

In [ ]:
def load_slippage(r):
    exchange, _, symbol = r.name
    try:
        slippage = pd.read_pickle(f"data/bid_ask_spreads_1h/{exchange}/{symbol}.pkl")
    except FileNotFoundError:
        r["slippage"] = np.nan
        return r
    
    slippage["sum"] = slippage["bid_amount"] + slippage["ask_amount"]
    slippage = slippage.sort_values(["timestamp", "sum"], ascending=False)
    slippage = slippage.groupby("timestamp", as_index=False).apply(lambda g: g.iloc[:3])
    slippage["idx"] = slippage.groupby("timestamp").cumcount() + 1
    slippage = slippage.pivot(index="timestamp", columns="idx")
    slippage = slippage.droplevel(1, axis=1)
    slippage = slippage.reindex(pickle.loads(r["ohlcv"]).open_time)

    r["slippage"] = pickle.dumps(slippage)
    return r

slippages = ohlcvs.apply(load_slippage, axis=1)[["slippage"]].dropna()

In [ ]:
ba_spreads = app.data_prep.dummy.dummy_bid_ask_spreads(ohlcvs, 0.002, force_default=True)
ba_spreads

In [ ]:
ba_spreads.sort_values("avg_slippage", ascending=True).tail(20)

In [ ]:
fee_info = app.data_prep.get_fee_info(refresh=False, identifiers=[identifier])
fee_info

In [ ]:
spreads = app.data_prep.create_spreads(ohlcvs, fee_info, bas=ba_spreads, bas_type="mean")

In [ ]:
spreads

In [ ]:
pd.concat([pickle.loads(spreads.iloc[0].spread).underlying_col(x) for x in ["bas", "bid_amount", "ask_amount"]], axis=1).iloc[550]

In [ ]:
with open("data/hourly_sum_slippage.pkl", "rb") as f:
    spreads = pickle.load(f)

In [ ]:
spreads.to_pickle("data/hourly_sum_slippage.pkl")

In [ ]:
spreads

In [ ]:
arr = np.stack(
    spreads.apply(
        lambda s: pd.concat(
            [pickle.loads(s.spread).underlying_col(x) for x in ["bas", "bid_amount", "ask_amount"]], axis=1
        ),
        axis=1,
    ),
    axis=1,
)

In [ ]:
arr.shape

In [ ]:
arr[550, 0, [[0, 6, 12], [3, 9, 15]]].T

In [ ]:
list(np.array([[0, 6, 12], [3, 9, 15]]))

In [ ]:
np.concatenate((np.array([1, 2]), np.array([3, 4])))

In [ ]:
np.nan_to_num(arr[550, 0].T, nan=0)

In [ ]:
from numba import njit
@njit
def foo3(arr):
    print(arr[550, 0, [0, 6, 12]].T)
foo3(arr)

In [ ]:
from numba import njit


@njit
def foo2(arr):
    trade_sizes = np.array([-214, 283])
    slippage_ = np.array([0.0, 0.0])
    arr = arr[550, 0]
    for leg in range(2):
        side_idx = 1 if trade_sizes[leg] > 0 else 0
        order_amt = abs(trade_sizes[leg])
        
        for lv in range(3):
            slip = arr[leg*3 + lv]
            if np.isnan(slip):
                continue
            slip_amt = arr[leg*3 + 6*side_idx + 6+lv]
            leftover = max(order_amt - slip_amt, 0)
            
            slippage_[leg] += ((order_amt - leftover) / order_amt) * slip
            
            if leftover == 0:
                break
        
        if leftover > 0:
            slippage_[leg] *= 1.5
            
        print(leg, slippage_[leg])
            
foo2(arr)

In [ ]:
res = app.BacktestRunner(
    log_dir=os.path.join(
        os.getenv("ACTIVE_DEV_PATH", "/home/stefano/development/active_dev"), "logs", "spreads-arb-v2"
    ),
    use_slippage=True,
    slippage_type="mean",
    use_funding_rate=False,
    profitable_only=True,
    z_score_thresholds=(0, 1),
    z_score_period=500,
).run_chained(spreads)

In [ ]:
res.zscores

In [ ]:
res.close_prices

In [ ]:
res.orders

In [ ]:
res.trades

In [ ]:
batch = res.batch_run([0.001, 0.0015, 0.002, 0.0025, 0.003, 0.0035, 0.004])

In [ ]:
batch

In [ ]:
res.aggregate_stats()

In [ ]:
res.print_all_trades()

In [ ]:
positions = (
    res.portfolio.assets()
    .set_axis(range(len(spreads) * 2), axis=1)
    .melt(var_name="col", ignore_index=False)
    .pipe(lambda df: df[df.value != 0])
    .reset_index()
)

positions_0 = positions[positions.col < len(spreads)].rename(columns={"col": "col_0", "value": "value_0"})
positions_1 = positions[positions.col >= len(spreads)].rename(columns={"col": "col_1", "value": "value_1"})
positions = positions_0.merge(positions_1).sort_values("open_time")
positions = (
    positions.merge(res.close_prices, left_on=["open_time", "col_0"], right_on=["Timestamp", "col"], how="right")
    .rename(columns={"close": "close_0"})
    .drop(columns=["col", "open_time"])
    .drop_duplicates(["Timestamp", "col_0", "col_1"])
)
# positions = (
#     positions.merge(res.close_prices, left_on=["Timestamp", "col_1"], right_on=["Timestamp", "col"], how="right")
#     .rename(columns={"close": "close_1"})
#     .drop(columns="col")
#     .drop_duplicates("Timestamp")
# )
# positions = positions.merge(
#     res.zscores(period=res.z_score_period), left_on=["Timestamp", "col_0"], right_on=["open_time", "col"], how="right"
# ).drop(columns="col")

In [ ]:
positions

In [ ]:
positions.merge(res.close_prices, left_on=["Timestamp", "col_1"], right_on=["Timestamp", "col"], how="right")

In [ ]:
res.plot()